## Definisi Masalah

Tenggelamnya kapal RMS Titanic adalah salah satu tragedi yang paling terkenal. Pada tanggal 15 April 1912, dalam pelayaran perdana ny, kapal Titanic tenggelam setelah bertabrakan dengan sebuah gunung es, mengakibatkan 1502 korban jiwa dari 2224 penumpan dan awak kapal. Tragedi ini sangat mengguncang komunitas internasional dan mendorong untuk membuat peraturan keselamatan yang lebih baik di dalam kapal dan pelayaran.

Salah satu alasan kapal Titanic mengakibatkan korban jiwa yang relatif banyak diakibatkan karena tidak cukupnya jumlah kapal penyelemat (skoci) bagi para penumpang dan awak kapal. Meskipun ada yang beruntung selamat pada saat proses penenggelaman, kelompok orang yang lebih banyak selamat dibandingkan yang lain antara lain para wanita, anak - anak, dan para kelas atas.

Dalam tugas ini, kita meminta anda untuk menyelesaikan analisis mengenai kriteria seperti apa orang yang selamat dari tragedi kapal Titanic. Khususnya, kita meminta anda untuk menerapkan <i>machine learning</i> untuk memprediksi penumpang mana saja yang selamat dari tragedi tersebut.

## Requirement

1. Tim pengajar telah menyiapkan data yang sudah dieksplorasi dan dibersihkan sehingga siap untuk dimasukkan ke dalam model
2. Tim pengajar telah menyiapkan 2 model yang sudah di-<i>training</i> dengan menggunakan algoritma <i>Random Forest</i> dan <i>Adaboost</i>
3. Buatlah model FCNN untuk memperkuat analisis dan prediksi anda
5. Buatlah model <i>Ensemble</i> yang menggabungkan model <i>Random Forest</i>, <i>Adaboost</i>, dan <i>FCNN</i>. Anda dapat memilih untuk menggunakan <i>Majority Vote</i> atau <i>Stacking</i>
6. Buatlah tabel perbandingan mengenai performa keempat model yang telah dibuat

## Library Import

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
%matplotlib inline

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split

c:\users\benedict\miniconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


## Import Dataset

In [2]:
df = pd.read_csv("./data/titanic.csv")
df.shape

(891, 8)

Berikut adalah deskripsi dari masing - masing kolom:
- survival = Selamat atau tidaknya penumpang yang bersangkutan (0 = Tidak, 1 = Selamat)
- pclass = Kelas tiket penumpang (1 = 1st, 2 = 2nd, 3 = 3rd)
- sex = Gender
- Age = Umur dalam tahun
- sibsp = Jumlah saudara kandung/pasangan yang bersama dengan penumpang bersangkutan di atas kapal
- parch = Jumlah orang tua/anak - anak yang bersama dengan penumpang bersangkutan di atsa kapal
- fare = Jumlah biaya perjalanan penumpang
- cabin = Nomor kabin
- embarked = Nama pelabuhan tempat penumpang berangkat

In [3]:
df.head()

,Age,Embarked,Fare,Parch,Pclass,Sex,SibSp,Survived
0,22,S,7.2500,0,3,male,1,0.0
1,38,C,71.2833,0,1,female,1,1.0
2,26,S,7.9250,0,3,female,0,1.0
3,35,S,53.1000,0,1,female,1,1.0
4,35,S,8.0500,0,3,male,0,0.0


## Preprocessing

In [4]:
numeric_data = df.loc[:, ["Age", "Fare", "Parch", "SibSp"]].copy()
categorical_data = df.loc[:, ["Embarked", "Pclass", "Sex",]].copy()

In [5]:
# Standard Scaling numerical data
sc = StandardScaler()
numeric_data = sc.fit_transform(numeric_data)
numeric_data.shape

(891, 4)

In [6]:
# Label Binarizer categorical data
embarked_dummy = pd.get_dummies(categorical_data["Embarked"], prefix="embarked")
categorical_data = categorical_data.drop(["Embarked"], axis=1)
categorical_data = pd.concat([categorical_data, embarked_dummy], axis=1)

pclass_dummy = pd.get_dummies(categorical_data["Pclass"], prefix="pclass")
categorical_data = categorical_data.drop(["Pclass"], axis=1)
categorical_data = pd.concat([categorical_data, pclass_dummy], axis=1)

sex_dummy = pd.get_dummies(categorical_data["Sex"], prefix="sex")
categorical_data = categorical_data.drop(["Sex"], axis=1)
categorical_data = pd.concat([categorical_data, sex_dummy], axis=1)

categorical_data.head()

,embarked_C,embarked_Q,embarked_S,pclass_1,pclass_2,pclass_3,sex_female,sex_male
0,0,0,1,0,0,1,0,1
1,1,0,0,1,0,0,1,0
2,0,0,1,0,0,1,1,0
3,0,0,1,1,0,0,1,0
4,0,0,1,0,0,1,0,1


In [7]:
# Combine the preprocessed numerical data & categorical data
df = df.loc[:, ["Age", "Fare", "Parch", "SibSp", "Survived"]].copy()
df.loc[:, "Age":"SibSp"] = numeric_data
df = pd.concat([df, categorical_data], axis=1)
df.head()

,Age,Fare,Parch,SibSp,Survived,embarked_C,embarked_Q,embarked_S,pclass_1,pclass_2,pclass_3,sex_female,sex_male
0,-0.510769,-0.502445,-0.473674,0.432793,0.0,0,0,1,0,0,1,0,1
1,0.579769,0.786845,-0.473674,0.432793,1.0,1,0,0,1,0,0,1,0
2,-0.238134,-0.488854,-0.473674,-0.474545,1.0,0,0,1,0,0,1,1,0
3,0.375293,0.420730,-0.473674,0.432793,1.0,0,0,1,1,0,0,1,0
4,0.375293,-0.486337,-0.473674,-0.474545,0.0,0,0,1,0,0,1,0,1


In [8]:
# Train test split
X = df.drop(["Survived"], axis=1)
y = df["Survived"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=123)

## #1 Model - Random Forest Classifier

In [9]:
filename = "./model/rf.sav"
rf_model = pickle.load(open(filename, 'rb'))
rf_model

c:\users\benedict\miniconda3\lib\site-packages\sklearn\base.py:311: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.19.0 when using version 0.19.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
c:\users\benedict\miniconda3\lib\site-packages\sklearn\base.py:311: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.19.0 when using version 0.19.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
            oob_score=False, random_state=123, verbose=1, warm_start=False)

## #2 Model - AdaBoost Classifier

One of many boosting algorithm

In [10]:
filename = "./model/ad.sav"
ab_model = pickle.load(open(filename, 'rb'))
ab_model

c:\users\benedict\miniconda3\lib\site-packages\sklearn\base.py:311: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.19.0 when using version 0.19.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
c:\users\benedict\miniconda3\lib\site-packages\sklearn\base.py:311: UserWarning: Trying to unpickle estimator AdaBoostClassifier from version 0.19.0 when using version 0.19.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=0.1, n_estimators=1000, random_state=123)

## #3 Model - FCNN (deep learning) (50 point)

Buatlah model FCNN dengan menggunakan Keras yang dapat memprediksi target (survival) dari kasus di atas. Anda dapat membuat <i>neural network</i> dengan menggunakan 2 <i>hidden layer</i>. 

NB: Jangan lupa untuk meng-<i>import</i> <i>Keras library</i> terlebih dahulu

In [11]:
X_train.shape

(801, 12)

In [13]:
import keras
dir(keras)

Using TensorFlow backend.


['Input',
 'Model',
 'Sequential',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 'absolute_import',
 'activations',
 'applications',
 'backend',
 'callbacks',
 'constraints',
 'datasets',
 'engine',
 'initializers',
 'layers',
 'legacy',
 'losses',
 'metrics',
 'models',
 'optimizers',
 'preprocessing',
 'regularizers',
 'utils',
 'wrappers']

In [15]:
"Import Keras"
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

"Bikin Baseline model"

def keras_model():
    model = Sequential()
    model.add(Dense(60, input_dim=12, kernel_initializer='normal',activation='relu'))
    model.add(Dense(30, kernel_initializer='normal',activation='relu'))
    model.add(Dense(1, kernel_initializer='normal',activation='sigmoid'))
    # Compile Model
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model

fcnn_model = KerasClassifier(build_fn=keras_model, epochs=200, batch_size=64, verbose=1, validation_data=(X_test,y_test));
fcnn_model.fit(X_train, y_train)


#fcnn_model.fit(X_train,y_train, batch_size=64, epochs=200, verbose=1,validation_data=(X_test,y_test));

Train on 801 samples, validate on 90 samples
Epoch 1/200
801/801 [==============================] - 0s 360us/step - loss: 0.6896 - acc: 0.6055 - val_loss: 0.6806 - val_acc: 0.6889
Epoch 2/200
801/801 [==============================] - 0s 20us/step - loss: 0.6781 - acc: 0.6142 - val_loss: 0.6567 - val_acc: 0.7333
Epoch 3/200
801/801 [==============================] - 0s 27us/step - loss: 0.6526 - acc: 0.6604 - val_loss: 0.6057 - val_acc: 0.8000
Epoch 4/200
801/801 [==============================] - 0s 21us/step - loss: 0.6086 - acc: 0.7054 - val_loss: 0.5310 - val_acc: 0.8222
Epoch 5/200
801/801 [==============================] - 0s 24us/step - loss: 0.5552 - acc: 0.7303 - val_loss: 0.4599 - val_acc: 0.8444
Epoch 6/200
801/801 [==============================] - 0s 24us/step - loss: 0.5124 - acc: 0.7740 - val_loss: 0.4197 - val_acc: 0.8667
Epoch 7/200
801/801 [==============================] - 0s 25us/step - loss: 0.4788 - acc: 0.7978 - val_loss: 0.3853 - val_acc: 0.8444
Epoch 8/200
801/

Epoch 61/200
801/801 [==============================] - 0s 36us/step - loss: 0.3803 - acc: 0.8477 - val_loss: 0.3626 - val_acc: 0.8778
Epoch 62/200
801/801 [==============================] - 0s 30us/step - loss: 0.3789 - acc: 0.8439 - val_loss: 0.3558 - val_acc: 0.8778
Epoch 63/200
801/801 [==============================] - 0s 30us/step - loss: 0.3787 - acc: 0.8414 - val_loss: 0.3680 - val_acc: 0.8667
Epoch 64/200
801/801 [==============================] - 0s 30us/step - loss: 0.3782 - acc: 0.8427 - val_loss: 0.3558 - val_acc: 0.8889
Epoch 65/200
801/801 [==============================] - 0s 27us/step - loss: 0.3780 - acc: 0.8427 - val_loss: 0.3555 - val_acc: 0.8889
Epoch 66/200
801/801 [==============================] - 0s 29us/step - loss: 0.3782 - acc: 0.8439 - val_loss: 0.3662 - val_acc: 0.8667
Epoch 67/200
801/801 [==============================] - 0s 35us/step - loss: 0.3767 - acc: 0.8402 - val_loss: 0.3561 - val_acc: 0.8889
Epoch 68/200
801/801 [==============================] -

801/801 [==============================] - 0s 26us/step - loss: 0.3573 - acc: 0.8514 - val_loss: 0.3667 - val_acc: 0.8444
Epoch 122/200
801/801 [==============================] - 0s 24us/step - loss: 0.3572 - acc: 0.8439 - val_loss: 0.3632 - val_acc: 0.8444
Epoch 123/200
801/801 [==============================] - 0s 29us/step - loss: 0.3565 - acc: 0.8527 - val_loss: 0.3566 - val_acc: 0.8778
Epoch 124/200
801/801 [==============================] - 0s 24us/step - loss: 0.3552 - acc: 0.8514 - val_loss: 0.3642 - val_acc: 0.8556
Epoch 125/200
801/801 [==============================] - 0s 25us/step - loss: 0.3555 - acc: 0.8502 - val_loss: 0.3654 - val_acc: 0.8667
Epoch 126/200
801/801 [==============================] - 0s 25us/step - loss: 0.3549 - acc: 0.8464 - val_loss: 0.3684 - val_acc: 0.8333
Epoch 127/200
801/801 [==============================] - 0s 24us/step - loss: 0.3554 - acc: 0.8577 - val_loss: 0.3629 - val_acc: 0.8444
Epoch 128/200
801/801 [==============================] - 0s 22

801/801 [==============================] - 0s 27us/step - loss: 0.3389 - acc: 0.8652 - val_loss: 0.3659 - val_acc: 0.8667
Epoch 182/200
801/801 [==============================] - 0s 27us/step - loss: 0.3389 - acc: 0.8564 - val_loss: 0.3881 - val_acc: 0.8667
Epoch 183/200
801/801 [==============================] - 0s 22us/step - loss: 0.3388 - acc: 0.8564 - val_loss: 0.3641 - val_acc: 0.8778
Epoch 184/200
801/801 [==============================] - 0s 25us/step - loss: 0.3425 - acc: 0.8502 - val_loss: 0.3919 - val_acc: 0.8444
Epoch 185/200
801/801 [==============================] - 0s 24us/step - loss: 0.3390 - acc: 0.8652 - val_loss: 0.3689 - val_acc: 0.8889
Epoch 186/200
801/801 [==============================] - 0s 22us/step - loss: 0.3358 - acc: 0.8627 - val_loss: 0.3794 - val_acc: 0.8778
Epoch 187/200
801/801 [==============================] - 0s 22us/step - loss: 0.3361 - acc: 0.8589 - val_loss: 0.3767 - val_acc: 0.8444
Epoch 188/200
801/801 [==============================] - 0s 21

## #4 Model - Ensemble (40 point)

Buatlah model <i>Ensemble</i> dengan menggabungkan model #1, #2, dan #3. Anda dapat menggunakan metode <i>Majority Vote</i> (class <i>Majority Voter</i> sudah kami sediakan) atau metode <i>Stacking</i>

In [16]:
# Majority Voter Class
from sklearn.base import BaseEstimator
from sklearn.base import ClassifierMixin
from sklearn.preprocessing import LabelEncoder
from sklearn.externals import six
from sklearn.base import clone
from sklearn.pipeline import _name_estimators
import numpy as np
import operator
class MajorityVoteClassifier(BaseEstimator, 
                             ClassifierMixin):
    """ A majority vote ensemble classifier
    Parameters
    ----------
    classifiers : array-like, shape = [n_classifiers]
      Different classifiers for the ensemble
    vote : str, {'classlabel', 'probability'} (default='label')
      If 'classlabel' the prediction is based on the argmax of
        class labels. Else if 'probability', the argmax of
        the sum of probabilities is used to predict the class label
        (recommended for calibrated classifiers).
    weights : array-like, shape = [n_classifiers], optional (default=None)
      If a list of `int` or `float` values are provided, the classifiers
      are weighted by importance; Uses uniform weights if `weights=None`.
    """
    def __init__(self, classifiers, vote='classlabel', weights=None):
        self.classifiers = classifiers
        self.named_classifiers = {key: value for key, value
                                  in _name_estimators(classifiers)}
        self.vote = vote
        self.weights = weights
    def fit(self, X, y):
        """ Fit classifiers.
        Parameters
        ----------
        X : {array-like, sparse matrix}, shape = [n_samples, n_features]
            Matrix of training samples.
        y : array-like, shape = [n_samples]
            Vector of target class labels.
        Returns
        -------
        self : object
        """
        if self.vote not in ('probability', 'classlabel'):
            raise ValueError("vote must be 'probability' or 'classlabel'"
                             "; got (vote=%r)"
                             % self.vote)
        if self.weights and len(self.weights) != len(self.classifiers):
            raise ValueError('Number of classifiers and weights must be equal'
                             '; got %d weights, %d classifiers'
                             % (len(self.weights), len(self.classifiers)))
        # Use LabelEncoder to ensure class labels start with 0, which
        # is important for np.argmax call in self.predict
        self.lablenc_ = LabelEncoder()
        self.lablenc_.fit(y)
        self.classes_ = self.lablenc_.classes_
        self.classifiers_ = []
        for clf in self.classifiers:
            fitted_clf = clone(clf).fit(X, self.lablenc_.transform(y))
            self.classifiers_.append(fitted_clf)
        return self
    def predict(self, X):
        """ Predict class labels for X.
        Parameters
        ----------
        X : {array-like, sparse matrix}, shape = [n_samples, n_features]
            Matrix of training samples.
        Returns
        ----------
        maj_vote : array-like, shape = [n_samples]
            Predicted class labels.
            
        """
        if self.vote == 'probability':
            maj_vote = np.argmax(self.predict_proba(X), axis=1)
        else:  # 'classlabel' vote
            #  Collect results from clf.predict calls
            predictions = np.asarray([np.reshape(clf.predict(X), 90)
                                      for clf in self.classifiers]).T
            maj_vote = np.apply_along_axis(
                                      lambda x:
                                      np.argmax(np.bincount(x,
                                                weights=self.weights)),
                                      axis=1,
                                      arr=predictions.astype(np.int64))
        #maj_vote = self.lablenc_.inverse_transform(maj_vote)
        return maj_vote
    def predict_proba(self, X):
        """ Predict class probabilities for X.
        Parameters
        ----------
        X : {array-like, sparse matrix}, shape = [n_samples, n_features]
            Training vectors, where n_samples is the number of samples and
            n_features is the number of features.
        Returns
        ----------
        avg_proba : array-like, shape = [n_samples, n_classes]
            Weighted average probability for each class per sample.
        """
        probas = np.asarray([clf.predict_proba(X)
                             for clf in self.classifiers_])
        avg_proba = np.average(probas, axis=0, weights=self.weights)
        return avg_proba
    def get_params(self, deep=True):
        """ Get classifier parameter names for GridSearch"""
        if not deep:
            return super(MajorityVoteClassifier, self).get_params(deep=False)
        else:
            out = self.named_classifiers.copy()
            for name, step in six.iteritems(self.named_classifiers):
                for key, value in six.iteritems(step.get_params(deep=True)):
                    out['%s__%s' % (name, key)] = value
            return out

In [17]:
# Contoh pemakaian MajoritVoteClassifier

mjv_model = MajorityVoteClassifier(classifiers=[rf_model, ab_model, fcnn_model])

"""
    Anda tidak perlu men-training Majority Voter lagi karena model - model yang menjadi 
    komposisi Majority Voter sudah dilatih sebelumnya
"""

'\n    Anda tidak perlu men-training Majority Voter lagi karena model - model yang menjadi \n    komposisi Majority Voter sudah dilatih sebelumnya\n'

## Perbandingan Performa Model (10 point)

Sertakan tabel perbandingan performa model (anda dapat menggunakan F1 score).

In [18]:
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import f1_score


In [19]:
y_pred = mjv_model.predict(X_test)


90/90 [==============================] - 0s 311us/step


[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.0s finished


In [20]:
f1_score(y_test,y_pred)

0.7719298245614034